In [1]:
import uproot
import numpy as np
import sys
import h5py
import pickle

In [2]:
if '../../WatChMaL' not in sys.path:
    sys.path.append('../../WatChMaL')

from watchmal.dataset.DigiTruthMapping import DigiTruthMapping
from watchmal.dataset.h5_dataset import H5TrueDataset
from watchmal.dataset.cnn_mpmt.cnn_mpmt_dataset import CNNmPMTDataset

In [3]:
## Load Dataset

In [4]:
# Import test events from h5 file
data_path = "/fast_scratch/WatChMaL/data/IWCD_mPMT_Short_emgp0_E0to1000MeV_digihits.h5"
data_file = h5py.File(data_path, "r")

print(data_file.keys())

angles     = np.array(data_file['angles'])
energies   = np.array(data_file['energies'])
positions  = np.array(data_file['positions'])
labels     = np.array(data_file['labels'])
root_files = np.array(data_file['root_files'])
event_ids  = np.array(data_file['event_ids'])
vetos      = np.array(data_file['veto'])

<KeysViewHDF5 ['angles', 'energies', 'event_hits_index', 'event_ids', 'hit_charge', 'hit_pmt', 'hit_time', 'labels', 'positions', 'root_files', 'veto', 'veto2']>


In [5]:
idxs_path = '/fast_scratch/WatChMaL/data/IWCD_mPMT_Short_4_class_3M_emgp0_idxs.npz'
idxs = np.load(idxs_path, allow_pickle=True)

train_idxs = idxs['train_idxs']
val_idxs   = idxs['val_idxs']
test_idxs  = idxs['test_idxs']

In [6]:
print(root_files[test_idxs][0:2])

[b'/localscratch/prouse.56905527.0/WCSim/e-/E0to1000MeV/unif-pos-R400-y300cm/4pi-dir/IWCD_mPMT_Short_e-_E0to1000MeV_unif-pos-R400-y300cm_4pi-dir_3000evts_0.root'
 b'/localscratch/prouse.56905527.0/WCSim/e-/E0to1000MeV/unif-pos-R400-y300cm/4pi-dir/IWCD_mPMT_Short_e-_E0to1000MeV_unif-pos-R400-y300cm_4pi-dir_3000evts_0.root']


In [7]:
## Load vetos

In [30]:
test_vetos  = vetos[test_idxs]
test_labels = labels[test_idxs]

e_OD_veto   = (test_labels == 1) & (test_vetos)
mu_OD_veto  = (test_labels == 2) & (test_vetos)

In [9]:
## Load True Momenta

In [10]:
momenta = np.load('./prep_data/3M_momenta.npz', allow_pickle=True)

test_true_momenta = momenta['test_momenta']

In [11]:
## Load to_wall

In [12]:
to_wall = np.load('./prep_data/3M_d_to_wall.npz', allow_pickle=True)

test_to_wall = to_wall['test_d_to_wall'] / 100

In [13]:
## Load d_wall

In [14]:
d_wall = np.load('./prep_data/3M_d_wall.npz', allow_pickle=True)

test_d_wall = d_wall['test_d_wall'] / 100

In [15]:
## Load Data Mappings

In [16]:
dtm = DigiTruthMapping(dataset='./prep_data/data_for_truth.pkl', mcset='./prep_data/truth_for_data.pkl')
print(dtm.get_data_entry(10000))

9825


In [17]:
pion_dtm = DigiTruthMapping(dataset='./prep_data/pion_data_for_truth.pkl', mcset='./prep_data/pion_truth_for_data.pkl')
print(pion_dtm.get_data_entry(10000))

9572


In [18]:
## Load fitqun Flags

In [19]:
# Retrieve flags
gamma_file_data   = uproot.open('/fast_scratch/WatChMaL/data/IWCD_mPMT_Short_gamma_E0to1000MeV_unif-pos-R400-y300cm_4pi-dir.fiTQun.root')['fiTQun;1']
e_file_data       = uproot.open('/fast_scratch/WatChMaL/data/IWCD_mPMT_Short_e-_E0to1000MeV_unif-pos-R400-y300cm_4pi-dir.fiTQun.root')['fiTQun;1']
mu_file_data      = uproot.open('/fast_scratch/WatChMaL/data/IWCD_mPMT_Short_mu-_E0to1000MeV_unif-pos-R400-y300cm_4pi-dir.fiTQun.root')['fiTQun;1']
pion_file_data    = uproot.open('/fast_scratch/WatChMaL/data/IWCD_mPMT_Short_pi0_E0to1000MeV_unif-pos-R400-y300cm_4pi-dir.fiTQun.root')['fiTQun;1']

gamma_file_data['fq1rpcflg'].interpretation

AsJagged(AsDtype("('>i4', (7,))"))

In [20]:
gamma_fqpi0mom1 = gamma_file_data.arrays('fqpi0mom1')['fqpi0mom1']
gamma_fqpi0mom2 = gamma_file_data.arrays('fqpi0mom2')['fqpi0mom2']
gamma_fqpi0nll  = gamma_file_data.arrays('fqpi0nll')['fqpi0nll']
gamma_fqpi0mass = gamma_file_data.arrays('fqpi0mass')['fqpi0mass']

e_fqpi0mom1 = e_file_data.arrays('fqpi0mom1')['fqpi0mom1']
e_fqpi0mom2 = e_file_data.arrays('fqpi0mom2')['fqpi0mom2']
e_fqpi0nll  = e_file_data.arrays('fqpi0nll')['fqpi0nll']
e_fqpi0mass = e_file_data.arrays('fqpi0mass')['fqpi0mass']

mu_fqpi0mom1 = mu_file_data.arrays('fqpi0mom1')['fqpi0mom1']
mu_fqpi0mom2 = mu_file_data.arrays('fqpi0mom2')['fqpi0mom2']
mu_fqpi0nll  = mu_file_data.arrays('fqpi0nll')['fqpi0nll']
mu_fqpi0mass = mu_file_data.arrays('fqpi0mass')['fqpi0mass']

pion_fqpi0mom1 = pion_file_data.arrays('fqpi0mom1')['fqpi0mom1']
pion_fqpi0mom2 = pion_file_data.arrays('fqpi0mom2')['fqpi0mom2']
pion_fqpi0nll  = pion_file_data.arrays('fqpi0nll')['fqpi0nll']
pion_fqpi0mass = pion_file_data.arrays('fqpi0mass')['fqpi0mass']

In [21]:
print(np.array(pion_fqpi0mass[:, 0]).shape)

(1200000,)


In [22]:
truehits_filepath = '/data/WatChMaL/data/IWCD_mPMT_Short_emg_E0to1000MeV_truehits.h5'
truehits_dataset = H5TrueDataset(h5_path=truehits_filepath)

In [32]:
# find indices in true dataset for offset to fitqun files
print(truehits_dataset.labels.shape)
gamma_base_idx = np.where(truehits_dataset.labels == 0)[0][0]
e_base_idx     = np.where(truehits_dataset.labels == 1)[0][0]
mu_base_idx    = np.where(truehits_dataset.labels == 2)[0][0]
pion_base_idx  = np.where(labels == 3)[0][0]

print(gamma_base_idx)
print(e_base_idx)
print(mu_base_idx)
print(pion_base_idx)

(21000000,)
3000000
0
6000000
20613195


In [24]:
print(np.sort(np.where(truehits_dataset.labels == 2)[0]))

[6000000 6000001 6000002 ... 8999997 8999998 8999999]


In [25]:
gamma_flags = gamma_file_data.arrays('fq1rpcflg')['fq1rpcflg']
e_flags     = e_file_data.arrays('fq1rpcflg')['fq1rpcflg']
mu_flags    = mu_file_data.arrays('fq1rpcflg')['fq1rpcflg']
pion_flags  = pion_file_data.arrays('fq1rpcflg')['fq1rpcflg']

gamma_fq1rpcflg_1 = np.array(gamma_flags[:, 0, 1])
e_fq1rpcflg_1     = np.array(e_flags[:, 0, 1])
mu_fq1rpcflg_1    = np.array(mu_flags[:, 0, 1])
pion_fq1rpcflg_1  = np.array(pion_flags[:, 0, 1])

gamma_fq1rpcflg_2 = np.array(gamma_flags[:, 0, 2])
e_fq1rpcflg_2     = np.array(e_flags[:, 0, 2])
mu_fq1rpcflg_2    = np.array(mu_flags[:, 0, 2])
pion_fq1rpcflg_2  = np.array(pion_flags[:, 0, 2])

In [36]:
# Construct mapping from test set to fitqun
gamma_fq_indices = []
e_fq_indices     = []
mu_fq_indices    = []
pion_fq_indices  = []
for idx in test_idxs:
    particle_label = labels[idx]
    truehits_index = dtm.get_truth_entry(idx)
    
    if particle_label == 0:
        gamma_fq_indices.append(truehits_index % gamma_base_idx)
    elif particle_label == 1:
        e_fq_indices.append(truehits_index)
    elif particle_label == 2:
        mu_fq_indices.append(truehits_index % mu_base_idx)
    elif particle_label == 3:
        # Use pion map instead
        truehits_index = pion_dtm.get_truth_entry(idx % pion_base_idx)
        pion_fq_indices.append(truehits_index)
    
gamma_fq_indices = np.array(gamma_fq_indices)
e_fq_indices     = np.array(e_fq_indices)
mu_fq_indices    = np.array(mu_fq_indices)
pion_fq_indices  = np.array(pion_fq_indices)

In [39]:
test_fq1rpcflg_1 = np.concatenate((e_fq1rpcflg_1[e_fq_indices],
                                  mu_fq1rpcflg_1[mu_fq_indices],
                                  gamma_fq1rpcflg_1[gamma_fq_indices],
                                  pion_fq1rpcflg_1[pion_fq_indices]
                                )) != 0

test_fq1rpcflg_2 = np.concatenate((e_fq1rpcflg_2[e_fq_indices],
                                  mu_fq1rpcflg_2[mu_fq_indices],
                                  gamma_fq1rpcflg_2[gamma_fq_indices],
                                  pion_fq1rpcflg_2[pion_fq_indices]
                                )) != 0

In [40]:
## Define cuts

In [41]:
to_wall_cut = ((test_labels == 0) | (test_labels == 1) | (test_labels == 3)) & (test_to_wall < 0.63*np.log(test_true_momenta) - 2)
d_wall_cut  = test_d_wall < 0.5

In [42]:
print(np.max(test_d_wall))

2.9999810457229614


In [43]:
fq_comparison = to_wall_cut | d_wall_cut | mu_OD_veto | test_fq1rpcflg_1 | test_fq1rpcflg_2

In [44]:
print(np.bincount(to_wall_cut))
print(np.bincount(d_wall_cut))
print(np.bincount(mu_OD_veto))
print(np.bincount(test_fq1rpcflg_1))
print(np.bincount(test_fq1rpcflg_2))
print("####################")

print(np.bincount(fq_comparison))

[3523327 1148422]
[3049812 1621937]
[4157833  513916]
[3761803  909946]
[3985659  686090]
####################
[1983641 2688108]


In [45]:
print(test_idxs.shape)
print('#######################')
print(np.where(to_wall_cut)[0].shape)
print(np.where(d_wall_cut)[0].shape)
print(np.where(mu_OD_veto)[0].shape)
print(np.where(test_fq1rpcflg_1)[0].shape)
print(np.where(test_fq1rpcflg_2)[0].shape)

(4671749,)
#######################
(1148422,)
(1621937,)
(513916,)
(909946,)
(686090,)


In [46]:
print(fq_comparison.shape)
print(np.where(fq_comparison)[0].shape)

(4671749,)
(2688108,)


In [47]:
print(np.delete(test_idxs, np.where(fq_comparison)[0], 0).shape)

(1983641,)


In [48]:
fq_comparison_OD_veto = fq_comparison | e_OD_veto

In [49]:
cuts = {
        'fq_comparison'        : np.where(fq_comparison)[0],
        'fq_comparison_OD_veto': np.where(fq_comparison_OD_veto)[0],
        'to_wall_cut'          : np.where(to_wall_cut)[0],
        'd_wall_cut'           : np.where(d_wall_cut)[0],
        'e_OD_veto'            : np.where(e_OD_veto)[0],
        'mu_OD_veto'           : np.where(mu_OD_veto)[0],
        'fq1rpcflg_1'          : np.where(test_fq1rpcflg_1)[0],
        'fq1rpcflg_2'          : np.where(test_fq1rpcflg_2)[0]
        }

In [50]:
with open('./prep_data/4_class_3M_fitqun_cuts.pickle', 'wb') as handle:
    pickle.dump(cuts, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [51]:
fq_mapping = {
        'gamma_fq_indices': gamma_fq_indices,
        'e_fq_indices'    : e_fq_indices,
        'mu_fq_indices'   : mu_fq_indices,
        'pion_fq_indices' : pion_fq_indices,
        }

In [52]:
with open('./prep_data/4_class_3M_fitqun_mapping.pickle', 'wb') as handle:
    pickle.dump(fq_mapping, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [53]:
np.savez('./prep_data/3M_fitqun_cuts.npz', cuts=cuts)

In [54]:
fq_test_set = np.concatenate((np.ones_like(np.array(e_fq_indices))*1,
                              np.ones_like(np.array(mu_fq_indices))*2,
                              np.ones_like(np.array(gamma_fq_indices))*0
                             )
                            )